In [1]:
import nltk
import pandas as pd
import random
import numpy as np
import string
import math

from nltk import ngrams
from nltk.corpus import stopwords
from nltk.stem.snowball import FrenchStemmer
from nltk.tokenize import word_tokenize

from numpy import array
from collections import Counter
from scipy.sparse import csr_matrix

from french_lefff_lemmatizer.french_lefff_lemmatizer import FrenchLefffLemmatizer

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD

from gensim.test.utils import common_dictionary, common_corpus
from gensim.models import LsiModel
from gensim import corpora, models, utils
from gensim.test.utils import common_corpus, common_dictionary, get_tmpfile
from gensim.models import LsiModel
from gensim.corpora import Dictionary

import re

In [2]:
# Use spacy lib
# On https://spacy.io/

import spacy
nlp = spacy.load('fr')

In [3]:
##############
# Parameters #
##############

min_gram = 1
max_gram = 3

# To create ours partitions, we must first know the years which will be the limits
limit_years = [2007, 2010, 2014]

# Ignore words that appear at a frequency less than tresh_freq in the corpus
tresh_freq = 0.8


In [4]:
# Datas preprocessing methods.

# Lemmatisation without poncutations

stemmer = nltk.stem.snowball.FrenchStemmer()
fstw = stopwords.words('french')

# French Stop Words, extraits depuis le fichier stopwords-fr.txt + stopwords french de nltk
sourceFST = [x.replace('\n', '') for x in open('stopwords-fr.txt', mode="r", encoding="utf-8").readlines()]+fstw

# Based on ration of french and english stopwords
def isEnglish(article):
    total_fsw = len([x for x in article.split() if x in sourceFST])
    total_esw = len([x for x in article.split() if x in stopwords.words('english')])
    ratio = 100
    if total_fsw != 0:
        ratio = total_esw/total_fsw
    return ratio > 1 and total_esw > 3

def lemmatize(article):
    artiregex = re.sub(" [0-z][0-z] ", " ", article) # word of length < 2
    artiregex = artiregex.lower()
    artiregex = re.sub("(é|è|ê)", "e", artiregex)
    output = []
    outPonc = artiregex.translate(artiregex.maketrans("","", string.punctuation))
    outLem = nlp(outPonc)
    for token in outLem:
        if token.lemma_ not in sourceFST and [x for x in token.lemma_ if x not in "0123456789"] != []:
            output.append(token.lemma_)
    res = ' '.join(output)
    return res

In [5]:
# Data Reading
data = pd.read_csv('export_articles_EGC_2004_2018.csv', sep='\t', header=0)

In [6]:
# Let's process our corpus, and determine a limit to split it in partitions

# usable[] correspond to our corpus processed
# limits[] let us know when to delimit partitions
limits = []
usable = []

prev_year = data['year'][0]
numArti = 0
for i in range(0, len(data['abstract']), 1):
    #if not null, empty, or whatever (so if there is a abstract):
    if not isinstance(data['abstract'][i], float) and not isEnglish(data['abstract'][i]):
        text = data['abstract'][i]
        if not isinstance(data['title'][i], float):
            text += " "+data['title'][i]

        numArti+=1
        usable.append(stemmer.stem(lemmatize(text)))
        year = data['year'][i]
        if year != prev_year:
            prev_year = year
            if year in limit_years:
                limits.append(numArti-1)
limits.append(numArti-1)


In [7]:
# Display pre-processed datas

vectorizer = TfidfVectorizer(stop_words=sourceFST, use_idf=True, ngram_range=(min_gram, max_gram), max_df=tresh_freq)
tfidf = vectorizer.fit_transform(usable)

print("nombre d'articles =", len(usable))
print("nombre de mots =", len(tfidf.toarray()[0]))
print("limits =", limits)

usable[0]

/info/etu/m2/i140302/venv/lib/python3.5/site-packages/sklearn/feature_extraction/text.py:286: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['quelqu'] not in stop_words.
  sorted(inconsistent))


nombre d'articles = 991
nombre de mots = 133044
limits = [222, 467, 693, 990]


'plateforme objectif permettre citoyen danalyserpar euxmemer tweet politique devenement specifiqu francepour cas lelection presidentiell ideo2017 analyser quasitemps reel message candidat fournir principal caracteristiqueslusage lexiqu politique comparaison entrer candidat ideo2017   plateforme citoyen dediee lanalyse tweet evenement polit'

In [8]:
# Creation of partitions_tfidf[], which give us the TFIDF of each cluster of each partition
# partitions_tfidf[num_partition][num_doc][num_word]
# Beware, num_doc can't be equals to 1091 (max). You have partitions, so every doc aren't in every partitions
# num_word can be found via vectorizer.get_feature_name()
partitions_tfidf = []
beg = 0
for l in limits:
    last = l+1
    partitions_tfidf.append([list(x) for x in list(tfidf.toarray())[beg:last]])
    beg = l

In [9]:
len(partitions_tfidf)

4

In [ ]:
# INSERT KMEANS PROCESS HERE


In [ ]:
# INSERT QUALITY MEASURE HERE

In [10]:
# tf_of_your_word = tf[numDoc][strWord]
tf = []
for doc in usable:
    tf_doc = {}
    for word in vectorizer.get_feature_names():
        tf_doc[word] = doc.count(word)
    tf.append(tf_doc)

In [11]:
# Number total of words
# nb_total_word[numPartition]
nb_total_word = []
nb = 0

for numDoc in range(0, len(usable)):
    for word in vectorizer.get_feature_names():
        nb += tf[numDoc][word]
    if numDoc in limits:
        nb_total_word.append(nb)
        nb=0
    

In [12]:
nb_total_word

[104156, 111095, 98925, 131037]

In [13]:
tf[0]

{'application placement doffr': 0,
 'dan litteraturepour': 0,
 'letude motif': 0,
 'ditems meilleur exploitation': 0,
 'algorithmesde': 0,
 'niveau semantique appliqu': 0,
 'poindre methodologie': 0,
 'evaluation quantitatif biais': 0,
 'baser systeme vainqueur': 0,
 'timewindow': 0,
 'energetiqu nousinteresser': 0,
 'proposer system classification': 0,
 'successifsqui mesuree': 0,
 'lontologu': 0,
 'approcher musette': 0,
 'apport utiliser': 0,
 'evolutif application': 0,
 'numerisation document administratif': 0,
 'dolfaction fournir neuroscientifique': 0,
 'signatur information estdifficil': 0,
 'dexperts application proposees': 0,
 'foisd relation': 0,
 'maximisation em': 0,
 'presenter methode': 0,
 'constellation grâce': 0,
 'vrai cluster': 0,
 'dfc satisfaire': 0,
 'intervalle ferme': 0,
 'contexte applicatif recommandation': 0,
 'reecriture proposer': 0,
 'utiliseen combinaison': 0,
 'sarticul etape': 0,
 'heuristique ladaptation mapping': 0,
 'limiter dindice': 0,
 'dimension 

In [14]:
# nb_word[num_partition][word]
nb_word = []

word_in_this_parti = {}
for word in vectorizer.get_feature_names():
    word_in_this_parti[word] = 0

for numDoc in range(0, len(usable)):
    for word in vectorizer.get_feature_names():
        word_in_this_parti[word] += tf[numDoc][word]
    if numDoc in limits:
        nb_word.append(word_in_this_parti)
        word_in_this_parti = {}
        for word in vectorizer.get_feature_names():
            word_in_this_parti[word] = 0

In [15]:
len(nb_word)

4

In [16]:
# nb_word_by_cluster[numPartition][numCluster]
nb_word_by_cluster = []
for parti in partitions:
    nb_word_clus = []
    for cluster in parti:
        nb = 0
        for numDoc in cluster:
            for word in vectorizer.get_feature_name():
                nb += tf[numDoc][word]
        nb_word_clus.append(nb)
    nb_word_by_cluster.append(nb_word_clus)

NameError: name 'partitions' is not defined

In [ ]:
# Expected values, if nothing were dependant
# exp[numPartition][numCluster][numWord]
exp = []
for numParti in range(0, len(partitions)):
    exp_clus = []
    for numCluster in range(0, len(partitions[numParti]):
        exp_word = []
        for numWord in range(0, vectorizer.get_feature_names()):
            exp_word.append((nb_word[numParti][numWord] + nb_word_by_cluster[numPart][numCluster]) / nb_total_word[numParti])
        exp_cluster.append(exp_word)
    exp.append(exp_clus)


In [ ]:
# value_of_khi2 = khi2[numPartition][numCluster][word]
khi2 = []

for numParti in range(0, len(partitions)):
    khi2parti = []
    for numCluster in range(0, len(partitions[numParti])):
        khi2cluster = {}
        
        for word in vectorizer.get_feature_names():
            word_in_this_parti[word] = 0
            E = (nb_word[numParti][word] + nb_word_by_cluster[numParti][numCluster])/nb_total_word[numParti]
            N = 0
            for numDoc in partitions[numParti][numCluster]:
                N += tf[numDoc][word]
            khi2cluster.append(pow(N - E, 2)/E)        
        khi2parti.append(khi2cluster)
    khi2.append(khi2parti)

In [ ]:
# list of your labels = labels[numPartition][numCluster]
labels = []

for numPartition in range(0, len(nb_word_by_cluster)):
    for numCluster in range(0, len(nb_word_by_cluster[numPartition])):
        labels[numPartition][numCluster] = Counter(khi2[numPartition][numCluster]).most_commons(5)